In [ ]:
import numpy as np
import cv2
from scipy.spatial import distance as dist
from google.colab.patches import cv2_imshow
import imutils
import os

In [ ]:
MIN_CONF=0.3
NMS_THRES=0.3

USE_GPU=False
MIN_DISTANCE=50

In [ ]:
labelsPath ='/content/social-distance-detector/yolo-coco/coco.names'
LABELS = open(labelsPath).read().strip().split("\n")
weightPath='/content/social-distance-detector/yolo-coco/yolov3.weights'
configPath='/content/social-distance-detector/yolo-coco/yolov3.cfg'

In [ ]:
def detect_people(frame, net, ln, personIdx=0):

	(H, W) = frame.shape[:2]
	results = []

	blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
		swapRB=True, crop=False)
	net.setInput(blob)
	layerOutputs = net.forward(ln)

	boxes = []
	centroids = []
	confidences = []

	# loop over each of the layer outputs
	for output in layerOutputs:
		# loop over each of the detections
		for detection in output:
			# extract the class ID and confidence 
			# of the current object detection
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]


			if classID == personIdx and confidence > MIN_CONF:
	
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))


				boxes.append([x, y, int(width), int(height)])
				centroids.append((centerX, centerY))
				confidences.append(float(confidence))

	
	idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRES)

	if len(idxs) > 0:
		# loop over the indexes we are keeping
		for i in idxs.flatten():
			# extract the bounding box coordinates
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])

		
			r = (confidences[i], (x, y, x + w, y + h), centroids[i])
			results.append(r)


	return results

In [ ]:
# net=cv2.dnn.readNetFromDarknet(configPath,weightPath)
net=cv2.dnn_DetectionModel(configPath,weightPath)
# if USE_GPU:
#   net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
#   net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)


ln = net.getLayerNames()
# ln=net.getUnconnectedOutLayers()
# print(ln)

ln= [ln[i[0]-1] for i in net.getUnconnectedOutLayers()]

print("[INFO] accessing video stream...")
vs= cv2.VideoCapture('/content/social-distance-detector/pedestrians.mp4')
writer=None

while True:
  (grabbed,frame)= vs.read()

  if not grabbed:
    break

  frame= imutils.resize(frame,width=700)
  results=detect_people(frame,net,ln,personIdx=LABELS.index("person"))

  violate=set()
  
  if len(results)>=2:
    centroids=np.array([r[2] for r in results])
    D = dist.cdist(centroids,centroids,metric="euclidean")

    for i in range(0,D.shape[0]):
      for j in range(i+1,D.shape[1]):

        if D[i,j]<MIN_DISTANCE:
          violate.add(i)
          violate.add(j)

  for (i,(prob,bbox,centroid)) in enumerate(results):
    (startX,startY,endX,endY)=bbox
    (cX,cY)=centroid
    color=(0,255,0)

    if i in violate:
      color=(0,0,255)

    cv2.rectangle(frame,(startX,startY),(endX,endY),color,2)
    cv2.circle(frame,(cX,cY),5,color,1)
  
  text="Social Distance Violations: {}".format(len(violate))
  cv2.putText(frame,text,(10, frame.shape[0]-25),
              cv2.FONT_HERSHEY_SIMPLEX,0.85,(0,0,255),3)
  
  	# screen
	# if args["display"] > 0:
		# show the output frame
  cv2_imshow(frame)
  key = cv2.waitKey(1) & 0xFF

		# if the `q` key was pressed, break from the loop
  if key == ord("q"):
    break
  if writer is None:
    fourcc=cv2.VideoWriter_fourcc(*"MJPG")
    writer=cv2.VideoWriter('/content/out.avi', fourcc,25,
                           (frame.shape[1],frame.shape[0]),True)
    
  if writer is not None :
    writer.write(frame)



